In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

In [ ]:
# load in cms tsv data
cms_data = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/CRC_causal_discovery_data/CRC_CMS_pred_CMScaller.tsv', sep='\t')
cms_data 

In [ ]:
# count how many NaN in prediction column
cms_data['prediction'].isna().sum()

In [ ]:
# remove the NaN rows
cms_data = cms_data.dropna(subset=['prediction'])
cms_data

In [ ]:
data_kernel = pd.read_csv('/home/qiuaodon/Desktop/project_data_new/crc_100_bootstrap/outputs_PAGs_BS/heatmaps/quantile_patient_clusters_0.2.csv', sep='\t')

In [ ]:
# divide data in to 2 columns
data_kernel['patient'] = data_kernel['Patient,"Cluster"'].str.split(',').str[0]
data_kernel['cluster'] = data_kernel['Patient,"Cluster"'].str.split(',').str[1]
# remove the first column
data_kernel = data_kernel.drop(columns=['Patient,"Cluster"'])
data_kernel

In [ ]:
# remove the _bootstrap from the patient column
data_kernel['patient'] = data_kernel['patient'].str.replace('_bootstrap', '')
data_kernel

In [ ]:
# # only keep the rows also in cms_data
# data_kernel = data_kernel[data_kernel['patient'].isin(cms_data['Sample'])]
# data_kernel

# 

In [ ]:
# get the prediction column from cms_data to data_kernel matching by patient
data_kernel['prediction'] = data_kernel['patient'].map(cms_data.set_index('Sample')['prediction'])
data_kernel

In [ ]:
# if the prediction is NaN, make it 'NA'
data_kernel['prediction'] = data_kernel['prediction'].fillna('NA')
data_kernel

In [ ]:

# Plot 1: Distribution of prediction in each cluster
sns.histplot(data=data_kernel, x='cluster', hue='prediction', multiple='stack')
# put NA to the last and gray color
plt.xlabel("Cluster", fontsize=12)  # Adjust x-axis label font size
plt.ylabel("Count", fontsize=12)     # Adjust y-axis label font size
plt.title("Distribution of Prediction in Each Cluster", fontsize=12)  # Adjust title font size
plt.show()

# Plot 2: Distribution of prediction in each cluster
sns.histplot(data=data_kernel, x='prediction', hue='cluster', multiple='stack')
plt.xlabel("Prediction", fontsize=12)  # Adjust x-axis label font size
plt.ylabel("Count", fontsize=12)       # Adjust y-axis label font size
plt.title("Distribution of Cluster in Each Prediction", fontsize=12)  # Adjust title font size
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure "NA" is the last category only in the 'prediction' column for the first plot
data_kernel['prediction'] = pd.Categorical(data_kernel['prediction'], 
                                           categories=[cat for cat in data_kernel['prediction'].unique() if cat != 'NA'] + ['NA'],
                                           ordered=True)

# Define a color palette that assigns specific colors, with gray for "NA"
unique_predictions = data_kernel['prediction'].cat.categories
prediction_palette = sns.color_palette("Set2", len(unique_predictions) - 1) + [(0.5, 0.5, 0.5)]  # Gray for NA

# Plot 1: Distribution of prediction in each cluster with "NA" as the last category in gray
sns.histplot(data=data_kernel, x='cluster', hue='prediction', multiple='stack', palette=prediction_palette)
plt.xlabel("Cluster", fontsize=12)  # Adjust x-axis label font size
plt.ylabel("Count", fontsize=12)     # Adjust y-axis label font size
plt.title("Distribution of Prediction in Each Cluster", fontsize=12)  # Adjust title font size
plt.show()

# Filter out rows where 'cluster' is "NA" for the second plot
data_kernel_no_na = data_kernel[data_kernel['prediction'] != 'NA']

# Plot 2: Distribution of cluster in each prediction, excluding "NA"
sns.histplot(data=data_kernel_no_na, x='prediction', hue='cluster', multiple='stack')
plt.xlabel("Prediction", fontsize=12)  # Adjust x-axis label font size
plt.ylabel("Count", fontsize=12)       # Adjust y-axis label font size
plt.title("Distribution of Cluster in Each Prediction", fontsize=12)  # Adjust title font size
plt.show()
